In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct
import matplotlib.pyplot as plt

In [2]:
# set parameters
beta = 0.1     # exploration coefficient
alpha = 0.001   # observation noise variance
ls = 1.0        # length scale of the RBF kernel; larger values = more spatial generalization
scale = 0.0     # scale of Gumbel noise (set to 0 to turn off noise)
N = 30          # number of arms
T = 30          # number of trials to simulate
temp = 2        # softmax temperature
lambda_rbf = 1  # reward smoothness of env

# np.random.seed(0)

In [3]:
# Gaussian process prediction

def gp_predict(r, x, ls=1):
    # set up Gaussian process
    kernel = RBF(length_scale=ls) #+ DotProduct() ## you can also weight the terms
    gp = GaussianProcessRegressor(kernel=kernel, alpha=alpha, optimizer=None)
    gp.fit(x, r)
    m, sd = gp.predict(X, return_std=True)
    return m, sd

def ucb(m, sd, beta=beta):
    a = m+beta*sd
    print(beta+sd)
    return a

def softmax(x):
    e_x = np.exp(x/temp)
    prob_choose = e_x / e_x.sum()
    a = np.random.choice(X.reshape(N), p=prob_choose, size = (1))
    return a

p = np.linspace(0,1,N)

def R(a,lambda_rbf=1,seed=42):
    np.random.seed(seed)
    x_prime = int(np.random.choice(a, size=1)) #np.random.randint([a[0], a[-1]], size=1) #unimodal 
    print("Global max. : "+ str(x_prime))
    #max_reward = np.random.randint(70,90, size=1)
    #print("Max reward : "+str(max_reward[0]))
    r = np.exp(-(a-x_prime)**2/(2*lambda_rbf)) # probability distribution of r
    # r = r/np.sum(r)
    print(np.sum(r))
    plt.imshow(r.reshape(1,N), cmap='Reds')
    plt.title("Reward Distribution over the Environment")
    plt.xticks(ticks=np.arange(0,N), labels=np.arange(1,N+1))
    plt.show()
    return r #np.random.binomial(1,p[a])

def reward_kernel(a, lambda_rbf=1, seed=42):
    # choose a random global maximum 
    np.random.seed(seed)
    x_prime = int(np.random.choice(a, size=1)) #np.random.randint([a[0], a[-1]], size=1) #unimodal 
    print("Global max. : "+ str(x_prime))
    max_reward = np.random.randint(70,90, size=1)
    print("Max reward : "+str(max_reward[0]))
    r = max_reward*np.exp(-(a-x_prime)**2/(2*lambda_rbf))
    plt.imshow(r.reshape(1,N), cmap='Reds')
    plt.title("Reward Distribution over the Environment")
    plt.xticks(ticks=np.arange(0,N), labels=np.arange(1,N+1))
    plt.show()
    return r



In [4]:
# function to run 1 round of the 1D task 
%matplotlib qt
def run_gp(seed=1, T=T, N=N):
    
    reward = reward_kernel(X.reshape(N), lambda_rbf=4, seed = seed) #R(X.reshape(N), lambda_rbf=4, seed = seed) 
    r_t = np.zeros(T)
    actions_all = np.zeros(T)
    m = np.zeros((T,N))
    sd = np.zeros((T,N))
    for trial in range(T):
        a = int(softmax(ucb(m[trial], sd[trial], beta=1))) # sampling strategy + choice
        # print(a)
        actions_all[trial] = int(a)
        noise_reward = np.random.normal(loc = 0, scale=1, size= 1)
        r_t[trial] = reward[a-1] + noise_reward
        # r_t[trial] = np.random.binomial(1, reward[a-1])

        # print(actions_all[:trial+1])
        if trial < T-1:
            m[trial+1, :], sd[trial+1, :] = gp_predict(r=r_t[:trial+1], x=X[actions_all[:trial+1].astype(int)-1], ls=1)
        
    return actions_all, r_t, m, sd

# spatial location of arms (in 1D)

N = 1
X = np.linspace(1,N,N).reshape(N,1)

T = 100
actions_all, r_t, m, sd = run_gp(seed=42, T=T, N=30)
fig, ax = plt.subplots(2,2, figsize=(15,10))
ax = ax.flatten()
ax[0].plot(r_t)
ax[0].set_title('Reward over time')
ax[1].plot(np.arange(0,100), actions_all, 'ko')
ax[1].set_title('Actions over time')
ax[2].imshow(sd.T)
ax[2].set_title('Std')
ax[3].imshow(m.T)
ax[3].set_title('Mean')
plt.show()

ValueError: cannot reshape array of size 1 into shape (30,)

In [6]:
actions_all

array([15., 16.,  1., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11.])